In [1]:
from datetime import datetime, timedelta
from calendar_view.core import data
from calendar_view.config import style
from calendar_view.core.config import CalendarConfig
from calendar_view.calendar import Calendar
from calendar_view.core.event import Event
from calendar_view.core.event import EventStyles
from calendar_view.core.event import EventStyle
import requests
import os

In [2]:
def get_learning_maps_json():

    '''
    Uses the Rainfocus API to return the dictionary of all learning_maps in JSON format.
    '''
    
    url = "https://events.rainfocus.com/api/search"

    payload={'type': 'session',
    'catalogDisplay': 'list'}

    headers = {
    'rfapiprofileid': 'lEHxCfLvNr8N3euNXUyFcngGbQoLXL1U',
    'referer': 'https://www.ciscolive.com/'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return list(filter(lambda d: d.get('id') == 'learningmap', response.json()['attributes']))[0]

In [3]:
def get_learning_maps():

    '''
    Uses the get_learning_maps_json() API and return a dictionary of all learning map folders. 
    Each folder contains a list of (learning_map_name, learning_map_id) tuples.
    '''
    
    learning_maps_json = get_learning_maps_json()
    learning_maps = {}

    for value in learning_maps_json['values']:

        folder_name = value['id'].rstrip().replace('\u200e', '')
        learning_maps[folder_name] = []

        for child in value['child']['values']:
            learning_maps[folder_name].append((child['name'].replace('\u200b', '').replace('/', ' '), child['id']))



    return learning_maps

In [4]:
learning_maps = get_learning_maps()

In [6]:
def get_sessions(learning_map: str):
    '''
    Take a learning map and returns the associated sessions in a json object.
    '''

    url = "https://events.rainfocus.com/api/search"

    payload =   {'search.learningmap': learning_map,
                'type': 'session'}

    headers = {
    'rfapiprofileid': '0wnUkT1BBZK3JR2t3yc5huPwNQCS8C3n',
    'referer': 'https://www.ciscolive.com/'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    return response.json()

In [7]:
class Session():

    def __init__(self, session_json):
        self.type = session_json['type']
        try:
            self.start = datetime.strptime(session_json['times'][0]['utcStartTime'], '%Y/%m/%d %H:%M:%S') + timedelta(hours=1)
            self.end = datetime.strptime(session_json['times'][0]['utcEndTime'], '%Y/%m/%d %H:%M:%S') + timedelta(hours=1)
        except KeyError:
            self.start = 'Null'
            self.end = 'Null'
        self.name = session_json['title']
        self.id = session_json['code']
        match int(self.id.split('-')[1][0]):
            case 1: self.level = 'Introductory'
            case 2: self.level = 'Intermediate'
            case 3: self.level = 'Advanced'
            case 4: self.level = 'General'
        
    def __str__(self):
        return self.name

In [8]:
for FOLDER, COUPLES in learning_maps.items():

    FOLDER_PATH = './learning_maps/' + FOLDER

    if not os.path.exists(FOLDER_PATH):
        os.makedirs(FOLDER_PATH)

    for LEARNING_MAP_NAME, LEARNING_MAP in COUPLES:
        print(f'{LEARNING_MAP_NAME = }, {LEARNING_MAP = }')

        sessions_json = get_sessions(LEARNING_MAP)

        sessions = []

        for session in sessions_json['sectionList'][0]['items']:
            sessions.append(Session(session))

        style.hour_height = 500
        style.day_width = 1500
        style.event_notes_color = '#0D274D'
        style.title_font = style.image_font(250)
        style.hour_number_font = style.image_font(50)
        style.day_of_week_font = style.image_font(150)
        style.event_title_font = style.image_font(80)
        style.event_notes_font = style.image_font(60)


        config = data.CalendarConfig(
            lang='en',
            title=LEARNING_MAP_NAME,
            dates='2023-02-06 - 2023-02-10',
            show_date=True,
            mode='working_hours',
            title_vertical_align='top',
        )

        events = []

        for session in sessions:

            if session.type != 'Walk-in Lab':
                match session.level:
                        case 'Introductory': 
                            color = EventStyle(event_border=(116, 191, 75, 240), event_fill=(116, 191, 75, 180))
                        case 'Intermediate': 
                            color = EventStyle(event_border=(251, 171, 44, 240), event_fill=(251, 171, 44, 180))
                        case 'Advanced': 
                            color = EventStyle(event_border=(227, 36, 27, 240), event_fill=(227, 36, 27, 180))
                        case 'General': 
                            color = EventStyle(event_border=(0, 188, 235, 240), event_fill=(0, 188, 235, 180))
                        
                events.append(Event(day=session.start.strftime('%Y-%m-%d'), 
                                    start=session.start.strftime('%H:%M'), 
                                    end=session.end.strftime('%H:%M'), 
                                    title=session.id,
                                    notes=session.name,
                                    style=color))

        calendar = Calendar.build(config)
        calendar.add_events(events)
        calendar.save(FOLDER_PATH + '/' + LEARNING_MAP_NAME + ".png")

LEARNING_MAP_NAME = 'Full-Stack Observability', LEARNING_MAP = '1615903688656001adnz'
LEARNING_MAP_NAME = 'Internet and Cloud Intelligence', LEARNING_MAP = '1616599177462001at4I'
LEARNING_MAP_NAME = 'Performance Management for Enterprise Apps', LEARNING_MAP = '1615903688656002aj5K'
LEARNING_MAP_NAME = 'Cloud Automation', LEARNING_MAP = '1614364836859001PXpU'
LEARNING_MAP_NAME = 'Cloud Native Technologies', LEARNING_MAP = '1614364836859002PO0m'
LEARNING_MAP_NAME = 'Cloud Security', LEARNING_MAP = '1636127907316001zyqr'


KeyboardInterrupt: 